<a href="https://colab.research.google.com/github/chrisporras/xgdiss/blob/main/ChrisPorras_GradCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project 2023
## GradCAM plug n play

Machine Learning for Biomedical Data Science

Team Xtreme Gradient Dissenters

Members: Audrey Lee, Christian Porras, Joy Jiang

April 24, 2023

Using explainable AI framework `OmniXAI`

https://github.com/salesforce/OmniXAI

In [14]:
# # Colab not clearing RAM fix
# !wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
# !wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
# !wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
# !wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
# !apt install -qq libunwind8-dev
# !dpkg -i *.deb
# %env LD_PRELOAD=libtcmalloc.so
# !rm *.deb

## Install packages

In [1]:
!pip install -q omnixai # explainable ai framework
!pip install -q dash # web app, visualization
!pip install -q jupyter-dash # dash + jupyter notebooks/colab
!pip install -q dash_bootstrap_components # dash utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.9/534.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.0/758.0 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 14.6 MB/s eta 0:00:00


In [1]:
# NEED TO RESTART RUNTIME AFTER INSTALLING PACKAGES
import omnixai # WILL THROW ERROR IF RUNTIME NOT RESTARTED
# USE THIS TO TEST PACKAGE INSTALL

In [2]:
# for saving output htmls & accessing model .pt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Data
Run all cells to download data from kaggle and unzip into working directory.

In [3]:
!git clone https://github.com/chrisporras/xgdiss.git

Cloning into 'xgdiss'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 61 (delta 22), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (61/61), 3.69 MiB | 4.72 MiB/s, done.


In [4]:
# install Kaggle public api
! pip install -q kaggle
# Choose the kaggle.json file that you downloaded
! mkdir ~/.kaggle
! cp ./xgdiss/kaggle.json ~/.kaggle/
# Make directory named kaggle and copy kaggle.json file there.
!chmod 600 ~/.kaggle/kaggle.json
#Change the permissions of the file.
! kaggle datasets list

ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
salvatorerastelli/spotify-and-youtube                          Spotify and Youtube                                   9MB  2023-03-20 15:43:25          10128        354  1.0              
arnabchaki/data-science-salaries-2023                          Data Science Salaries 2023  💸                        25KB  2023-04-13 09:55:16           3904         89  1.0              
erdemtaha/cancer-data                                          Cancer Data                                          49KB  2023-03-22 07:57:00           4870        102  1.0              
evangower/premier-league-2022-2023                             Pr

In [5]:
!kaggle datasets download -d nickuzmenkov/strip-ai-256x256-png-tiles

100% 2.03G/2.03G [00:33<00:00, 119MB/s]
100% 2.03G/2.03G [00:33<00:00, 64.9MB/s]


In [6]:
!unzip strip-ai-256x256-png-tiles.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/b6f1e9_0-0012.png  
  inflating: train/b6f1e9_0-0013.png  
  inflating: train/b6f1e9_0-0015.png  
  inflating: train/b6f1e9_0-0016.png  
  inflating: train/b6f1e9_0-0017.png  
  inflating: train/b6f1e9_0-0018.png  
  inflating: train/b6f1e9_0-0019.png  
  inflating: train/b6f1e9_0-0022.png  
  inflating: train/b6f1e9_0-0023.png  
  inflating: train/b6f1e9_0-0024.png  
  inflating: train/b6f1e9_0-0025.png  
  inflating: train/b6f1e9_0-0026.png  
  inflating: train/b6f1e9_0-0027.png  
  inflating: train/b6f1e9_0-0029.png  
  inflating: train/b6f1e9_0-0030.png  
  inflating: train/b6f1e9_0-0031.png  
  inflating: train/b6f1e9_0-0032.png  
  inflating: train/b6f1e9_0-0033.png  
  inflating: train/b6f1e9_0-0037.png  
  inflating: train/b6f1e9_0-0038.png  
  inflating: train/b6f1e9_0-0040.png  
  inflating: train/b6f1e9_0-0071.png  
  inflating: train/b6f1e9_0-0072.png  
  inflating: train/b6f1e9_0-0073.png  
  inflating: 

## GradCAM

### Explainer pipeline functions

In [7]:
def get_xai(model, test, idx):
  import json
  import torch
  from torchvision import models, transforms
  from PIL import Image as PilImage
  from omnixai.preprocessing.image import Resize
  from omnixai.data.image import Image
  from omnixai.explainers.vision import VisionExplainer
  # from omnixai.visualization.dashboard import Dashboard
  # In this example, we consider an image classification task. We recommend using `Image`
  # to represent a batch of images. `Image` can be constructed from a numpy array or a Pillow
  # image. The following code loads a test image and resizes it to (256, 256).
  # img = Resize((256, 256)).transform(Image(PilImage.open('data/images/dog_cat.png').convert('RGB')))
  img = Image(PilImage.open(test['file_path'][idx]).convert('RGB'))
  # For visulization, the class names corresponding to the labels are also loaded.
  # with open('data/images/imagenet_class_index.json', 'r') as read_file:
  #     class_idx = json.load(read_file)
  #     idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
  device = "cuda" if torch.cuda.is_available() else "cpu"
  # The preprocessing function takes an `Image` instance as its input and outputs the
  # processed features that the ML model consumes. In this example, the `Image` object is
  # first converted into a torch tensor via the defined transform and sent to particular
  # device.
  train_ds_mean = torch.tensor([0.9113, 0.8299, 0.8212])
  train_ds_std = torch.tensor([0.1397, 0.2390, 0.3153])
  transform = transforms.Compose([
        transforms.ToTensor(),
        # transforms.Resize((224,224)),
        transforms.Normalize(train_ds_mean, train_ds_std)
  ])
  # preprocess = lambda ims: torch.stack([transform(im.to_pil()) for im in ims])

  # transform = transforms.Compose([
  #     transforms.Resize(256),
  #     transforms.CenterCrop(224),
  #     transforms.ToTensor(),
  #     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  # ])
  preprocess = lambda ims: torch.stack([transform(im.to_pil()) for im in ims]).to(device)
  # A ResNet model to explain
  # model = models.resnet50(pretrained=True).to(device)
  # The postprocessing function is a simple softmax function transforming the output logits
  # into class probabilities.
  postprocess = lambda logits: torch.nn.functional.softmax(logits, dim=1)
  # We now create a `VisionExplainer`, e.g., the selected explainers include Grad-CAM, LIME,
  # integreated-gradient. `params` in `VisionExplainer` allows us to set parameters for each
  # explainer applied here. For example, the target_layer for Grad-CAM#0 is set to the last
  # layer of `model.layer4`.
  explainer = VisionExplainer(
      explainers=["lime", "ig", "gradcam#0", "gradcam#1", "gradcam#2", "gradcam#3"],
      mode="classification",
      model=model,
      preprocess=preprocess,
      postprocess=postprocess,
      params={
          "gradcam#0": {"target_layer": model.layer4[-1]},
          "gradcam#1": {"target_layer": model.layer4[-2]},
          "gradcam#2": {"target_layer": model.layer4[-1]},
          "gradcam#3": {"target_layer": model.layer4[-2]},
      }
  )
  # Generate explanations given the test instances. The label to explain for the first two
  # Grad-CAM explainers is "bull_mastiff" (the top label) while the label for the second
  # Grad-CAM explainers is "tiger_cat" (label = 281).
  # compare gradcam for opposite class
  local_explanations = explainer.explain(
      img,
      params={
          "gradcam#2": {"y": [1-test['label_num'][idx]]},
          "gradcam#3": {"y": [1-test['label_num'][idx]]},
      }
  )
  # get predictions
  # predictions = local_explanations['predict'].get_explanations()
  # write combined html
  # with open(outpath, 'w') as f:
  #   for k in local_explanations.keys():
  #     fig = local_explanations[k]._plotly_figure(index=0, class_names='CL')
  #     f.write(fig.to_html(full_html=True, include_plotlyjs='cdn'))
  return local_explanations

In [8]:
# def _plotly_figure(self, index=0, class_names='CL', **kwargs):
#     import plotly.express as px
#     values = self.results["values"][index]
#     labels = self.results["labels"]
#     if labels is None:
#         fnames, scores = ["Predicted value"], [values]
#     else:
#         fnames, scores = labels[index], values
#         fnames = [class_names[f] for f in fnames] \
#             if class_names is not None else [str(f) for f in fnames]
#     fig = px.bar(
#         y=fnames[::-1],
#         x=scores[::-1],
#         orientation="h",
#         labels={"x": "Predicted value",
#                 "y": "Label" if labels is not None else "Target"},
#         title="",
#         color_discrete_map={True: "#008B8B", False: "#DC143C"},
#     )
#     return fig

In [9]:
def pickle_explanations(data,file_name):
  import pickle
  with open(file_name, 'wb') as file:
    pickle.dump(data, file)

In [10]:
def unpickle(file_name):
  import pickle
  # open a file, where you stored the pickled data
  file = open(file_name, 'rb')
  # dump information to that file
  exp = pickle.load(file)
  return exp

In [11]:
def write_html(data, outpath):
  with open(outpath, 'w') as f:
    for k in data.keys():
      fig = data[k]._plotly_figure(index=0, class_names='CL')
      f.write(fig.to_html(full_html=True, include_plotlyjs='cdn'))

### Run gradcam
Output:
1. Explanations object contianing images and scores from gradcam etc
2. Write .html with plotly visualizations to google drive
3. Save predictions and class probabilities in csv

In [12]:
import pandas as pd
import torch
lab = ['CE', 'LAA']
device = "cuda" if torch.cuda.is_available() else "cpu"
if device=='cpu':
  model = torch.load('/content/drive/MyDrive/XGD_explainable/230421_resnet18_mayo-clinic-tiled-2gb.pt',
                   map_location=torch.device('cpu'))
else:
  model = torch.load('/content/drive/MyDrive/XGD_explainable/230421_resnet18_mayo-clinic-tiled-2gb.pt')
test = pd.read_csv('/content/xgdiss/test.csv')

In [15]:
idx_list = [3,9]
for idx in idx_list:
  label = test['label'][idx]
  file_name = f'/content/drive/MyDrive/XGD_explainable/pkl/{label}/explanations-test-idx_{idx}-{label}.pkl'
  explanations = get_xai(model, test, idx)
  pickle_explanations(explanations, file_name)

  0%|          | 0/1000 [00:00<?, ?it/s]

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


  0%|          | 0/1000 [00:00<?, ?it/s]

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


In [51]:
# Visualize explanations
# exp = unpickle(file_name)
# for k in exp.keys():
#   fig = exp[k]._plotly_figure(index=0, class_names='CL')
#   fig.show()

## Working around memory capacity...

#### Playing with cache wipe

In [16]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import sys

In [ ]:
memory_usage_by_variable=pd.DataFrame({k:sys.getsizeof(v) for (k,v) in locals().items()},index=['Size'])
memory_usage_by_variable=memory_usage_by_variable.T
memory_usage_by_variable=memory_usage_by_variable.sort_values(by='Size',ascending=False).head(10)
memory_usage_by_variable.head()

NameError: ignored

In [ ]:
def obj_size_fmt(num):
  if num<10**3:
      return "{:.2f}{}".format(num,"B")
  elif ((num>=10**3)&(num<10**6)):
      return "{:.2f}{}".format(num/(1.024*10**3),"KB")
  elif ((num>=10**6)&(num<10**9)):
      return "{:.2f}{}".format(num/(1.024*10**6),"MB")
  else:
      return "{:.2f}{}".format(num/(1.024*10**9),"GB")


def memory_usage():
  memory_usage_by_variable=pd.DataFrame({k:sys.getsizeof(v)\
  for (k,v) in globals().items()},index=['Size'])
  memory_usage_by_variable=memory_usage_by_variable.T
  memory_usage_by_variable=memory_usage_by_variable\
  .sort_values(by='Size',ascending=False).head(10)
  memory_usage_by_variable['Size']=memory_usage_by_variable['Size'].apply(lambda x: obj_size_fmt(x))
  return memory_usage_by_variable

memory_usage()

,Size
test,1.32MB
_i4,3.92KB
_i1,849.00B
_i18,845.00B
_ii,845.00B
_i,816.00B
_i20,816.00B
_i21,812.00B
_i3,777.00B
memory_usage_by_variable,698.00B


In [ ]:
gc.collect()
gc.get_count()
(34, 0, 0)
memory_usage()

,Size
test,1.32MB
_i4,3.92KB
_,1.25KB
_21,1.25KB
_i1,849.00B
_i18,845.00B
_iii,845.00B
_ii,816.00B
_i20,816.00B
_i,812.00B


In [ ]:
!nvidia-smi

Wed Apr 26 01:45:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |   3253MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## SCRATCH

In [11]:
# test['prediction'] = np.zeros(test.shape[0]) 
# test['prob_CE'] = np.zeros(test.shape[0]) 
# test['prob_LAA'] = np.zeros(test.shape[0]) 
# test = test.iloc[:,1:] # drop unnamed: 0 col
### index of test ###
# idx = 0
# label = test['label'][idx]
# outpath = f'/content/drive/MyDrive/XGD_explainable/html/{label}/plotly-test-idx_{idx}-{label}.html'
# ## LOOP FOR ALL TEST SET ###
# ## Save 
# num_imgs = 0
# # num_imgs = test.shape[0] # ALL TEST IMAGES
# n = 50 # every 50 images, save test
# for idx in range(num_imgs):
#   print(f'Working on {idx}')
#   label = test['label'][idx]
#   outpath = f'/content/drive/MyDrive/XGD_explainable/html/{label}/plotly-test-idx_{idx}-{label}.html'
#   explanations = get_xai(model, test, idx)
#   # write_html(explanations, outpath)
#   predictions = explanations['predict'].get_explanations()
#   prob = predictions['values'][0]
#   prob_CE = prob[0]
#   prob_LAA = prob[1]
#   pred = lab[np.argmax(prob)]
#   test.loc[idx,'prediction'] = pred
#   test.loc[idx,'prob_CE'] = prob_CE
#   test.loc[idx,'prob_LAA'] = prob_LAA
#   # if idx % 50 == 0:
#   #   test.to_csv(f'/content/drive/MyDrive/XGD_explainable/html/test-{idx}.csv',
#   #           index=False)